<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Source" data-toc-modified-id="Data-Source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Source</a></span></li><li><span><a href="#Imports-and-Update-The-Latest-Data" data-toc-modified-id="Imports-and-Update-The-Latest-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports and Update The Latest Data</a></span></li><li><span><a href="#Historical-air-4-Thai-Data" data-toc-modified-id="Historical-air-4-Thai-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Historical air 4 Thai Data</a></span></li><li><span><a href="#Power-Plants" data-toc-modified-id="Power-Plants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Power Plants</a></span></li><li><span><a href="#Weather-Files" data-toc-modified-id="Weather-Files-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Weather Files</a></span><ul class="toc-item"><li><span><a href="#Fill-the-missing-value-in-Weather-Data" data-toc-modified-id="Fill-the-missing-value-in-Weather-Data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Fill the missing value in Weather Data</a></span></li></ul></li><li><span><a href="#Holiday-In-Thailand" data-toc-modified-id="Holiday-In-Thailand-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Holiday In Thailand</a></span></li><li><span><a href="#Hotspots-Data" data-toc-modified-id="Hotspots-Data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Hotspots Data</a></span></li></ul></div>

# Data Source 
1. Berekely Earth 'http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/'
2. Screaped Air4Thai Data 'http://air4thai.pcd.go.th/webV2/history/'
3. CDC Data 'https://www.cmuccdc.org/download_json/'
4. Old Air4Thai data from Thailand EPA

# Imports and Update The Latest Data

In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.read_data import *
from src.data.dl_weather import *
from src.gen_functions import *

In [2]:
b_folder='../data/pm25/'
a4th_folder='../data/air4thai_hourly/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'

In [3]:
# Data from Berkeley Earth Projects: 
download_b_data(data_folder='../data/pm25/', url='http://berkeleyearth.lbl.gov/air-quality/maps/cities/Thailand/')
get_city_info(data_folder='../data/pm25/')

100% [............................................................................] 801033 / 801033

In [4]:
download_cdc_data(station_url='https://www.cmuccdc.org/api/ccdc/stations', 
                  dl_url= 'https://www.cmuccdc.org/download_json/', 
                  data_folder='../data/cdc_data/')

  0%|                                                                                          | 0/404 [00:00<?, ?it/s]

number of stations 407


100%|████████████████████████████████████████████████████████████████████████████████| 404/404 [14:53<00:00,  2.21s/it]


In [5]:
update_last_air4Thai(url='http://air4thai.pcd.go.th/webV2/history/',data_folder='../data/air4thai_hourly/')

0it [00:00, ?it/s]

['02t', '03t', '05t', '08t', '10t', '11t', '12t', '13t', '14t', '16t', '17t', '18t', '19t', '20t', '21t', '22t', '24t', '25t', '26t', '27t', '28t', '29t', '30t', '31t', '32t', '33t', '34t', '35t', '36t', '37t', '38t', '39t', '40t', '41t', '42t', '43t', '44t', '46t', '47t', '50t', '52t', '53t', '54t', '57t', '58t', '59t', '60t', '61t', '62t', '63t', '67t', '68t', '69t', '70t', '71t', '72t', '73t', '74t', '75t', '76t', '77t', '79t', '80t', '81t', '82t', '83t', '84t', 'm1', 'm4', 'm8', 'm9', 'o10', 'o20', 'o22', 'o23', 'o24', 'o25', 'o26', 'o27', 'o28', 'o29']


81it [29:49, 22.10s/it]


# Historical air 4 Thai Data

In [99]:
# load stations information
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

# find stations in Chiangmai and parase that files
cm_station_ids = []
for i, stations in enumerate(station_info):
    if 'Chiang Mai' in stations['areaEN']:
        # ignore station that start with o
        if 'o' not in stations['stationID']:
            cm_station_ids.append(stations['stationID'])
            print(i)
print(cm_station_ids)

123
124
125
['35t', '36t', 'm9']


In [22]:
# parase historical data 
for station_id in cm_station_ids:
    # find all files that start with this stations
    p = Path(aqm_folder)
    filenames = []
    for i in p.glob('**/*.xlsx'):
        if station_id in i.name:
            filenames.append(str(i))
        
    # if filename exist load that file
    if len(filenames) >0:

        save_filename = aqm_folder + 'process/' + station_id + '.csv'
        print('save filename', save_filename)
        station_data = read_his_xl(filenames[0])
        #print(station_data.head())

        # save the data if the dataframe is not empty
        if len(station_data)> 0:
            station_data.to_csv(save_filename,index=True)

save filename ../data/aqm_hourly2/process/35t.csv
save filename ../data/aqm_hourly2/process/36t.csv


In [45]:
cm_data1 = pd.read_csv(aqm_folder + 'process/35t.csv')
cm_data1['datetime'] = pd.to_datetime(cm_data1['datetime'])
cm_data1 = cm_data1.set_index('datetime')
# keep only gas columns
cm_data1 = cm_data1[['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5']]

cm_data2 = pd.read_csv(aqm_folder + 'process/36t.csv')
cm_data2['datetime'] = pd.to_datetime(cm_data2['datetime'])
cm_data2 = cm_data2.set_index('datetime')


# Power Plants

In [92]:
url = 'https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B9%82%E0%B8%A3%E0%B8%87%E0%B9%84%E0%B8%9F%E0%B8%9F%E0%B9%89%E0%B8%B2%E0%B9%83%E0%B8%99%E0%B9%84%E0%B8%97%E0%B8%A2'
url = 'https://en.wikipedia.org/wiki/List_of_power_stations_in_Thailand'
table_list = pd.read_html(url)
len(table_list)
p_folder = '../data/power_plants/'
for i, table in enumerate(table_list):
    table.to_csv(p_folder + f'table_eng{i}.csv',index=False)

# Weather Files

In [91]:
folders = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/*/')
folders

['C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\bkk\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\chiang-mai\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\chiang-rai\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\kungming\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\luang-prabang\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\sikhottabong\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\tada_u\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\tak\\',
 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities\\yangong\\']

In [92]:
for folder in folders:
    city_name = Path(folder).name
    parent_folder = Path(folder).parent
    w_files = glob(folder + '/*.csv')
    print(city_name, len(w_files))
    filename = str(Path(folders[0]).parent) + '/' + city_name + '.csv'
    
    # concatenate all files 
    df_all = pd.DataFrame()
    for file in w_files:
        try:
            df = pd.read_csv(file)
            
        except:
            pass
        else:
            df_all = pd.concat([df_all, df], ignore_index=True)
    
    # drop missing value 
    df_all['datetime'] = pd.to_datetime(df_all['date'])
    df_all.drop('date',axis=1, inplace=True)
    df_all = df_all.sort_values('datetime')
    df_all = df_all.drop_duplicates('datetime', ignore_index=True)
    
    # save file
    df_all.to_csv(filename,index=False)

bkk 13
chiang-mai 36
chiang-rai 25
kungming 24
luang-prabang 24
sikhottabong 39
tada_u 24
tak 26
yangong 29


In [89]:
# find missing date for chiang-mai data 
df_all = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv')
df_all['datetime'] = pd.to_datetime(df_all['datetime'] )
# find exisiting date 
ex_date = df_all['datetime'].dt.strftime('%Y-%m-%d').unique()
ex_date = set(ex_date)
# calculate the datelist 
start_date = datetime(2000, 10, 1)
stop_date = datetime.now()
date_range = pd.date_range(start_date, stop_date).strftime('%Y-%m-%d')
missing_date = list(set(date_range).difference(ex_date))

In [87]:
with open('../data/weather_cities/weather_station_info.json','r') as f:
    station_dict_list = json.load(f)
   
i = 0 
city_json = station_dict_list[i]
print(city_json)

{'city_name': 'Mueang Chiang Mai', 'province': 'Chiang Mai', 'country': 'Thailand', 'station_name': 'Chiang Mai International Airport Station', 'specific_url': 'th/mueang-chiang-mai/', 'latitude': '18.8 °N', 'longitude': '98.97 °E'}


In [90]:
 bad_date_df = scrape_weather(city_json, date_range=missing_date, data_folder='C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/')

C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai/mueang-chiang-mai_weather.csv 2000-10-23


..\src\data\dl_weather.py:20: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 20 of the file ..\src\data\dl_weather.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(innerhtml)


##  Fill the missing value in Weather Data 

In [97]:
# weather data 
filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/chiang-mai.csv'
wea = pd.read_csv(filename)
wea['datetime']  = pd.to_datetime(wea['datetime'])
# roud datetiem to whole 30 mins 
wea['datetime'] = wea['datetime'].dt.round('30T')

dates = wea['datetime'].dropna().dt.date.unique()

# fill in the missing value
new_datetime = pd.date_range(start=dates[0], end=dates[-1], freq='30T') 
new_weather = pd.DataFrame(new_datetime, columns=['datetime'])
new_weather = new_weather.merge(wea, on='datetime',how='left')
print(new_weather.shape)

# remove strange T reading
lowest_t = 5 
idx = new_weather[new_weather['Temperature(C)']< lowest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

highest_t = 60
idx = new_weather[new_weather['Temperature(C)']> highest_t].index
new_weather.loc[idx,['Temperature(C)','Dew Point(C)','Humidity(%)']] = np.nan

new_weather = new_weather.fillna(method='ffill',limit=12)
new_weather = new_weather.fillna(method='bfill',limit=12)
new_weather = new_weather.set_index('datetime')
new_weather = new_weather.dropna(how='all').reset_index()
new_weather.to_csv(filename,index=False)

(333126, 11)


# Holiday In Thailand

https://www.timeanddate.com/holidays/thailand/2001

# Hotspots Data 

In [157]:
# load stations information for Chiangmai
station_info_file = aqm_folder + 'stations_locations.json'
with open(station_info_file, 'r',encoding="utf8") as f:
    station_info = json.load(f)
station_info = station_info['stations']

print(station_info[124])

{'stationID': '36t', 'nameTH': 'โรงเรียนยุพราชวิทยาลัย ', 'nameEN': 'Yupparaj Wittayalai School', 'areaTH': 'ต.ศรีภูมิ อ.เมือง, เชียงใหม่', 'areaEN': 'Si Phum, Meuang, Chiang Mai', 'stationType': 'GROUND', 'lat': '18.7909205', 'long': '98.9881062', 'LastUpdate': {'date': '2020-03-27', 'time': '03:00', 'PM25': {'value': '87', 'unit': 'µg/m³'}, 'PM10': {'value': '110', 'unit': 'µg/m³'}, 'O3': {'value': 'N/A', 'unit': 'ppb'}, 'CO': {'value': '1.09', 'unit': 'ppm'}, 'NO2': {'value': '-', 'unit': 'ppb'}, 'SO2': {'value': '2', 'unit': 'ppb'}, 'AQI': {'Level': '4', 'aqi': '192'}}}


In [158]:
# obtain the lat and long in km
lat_km = (merc_y(station_info[124]['lat'])/1E3 ).round()
long_km = (merc_x(station_info[124]['long'])/1E3).round()
print(lat_km,  long_km )

2117.0 11019.0


In [159]:
# file map folder 
m_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/M6/*.csv')
v_files = glob('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/fire_map/world_2000-2020/V1/*.csv')

In [148]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv'
shape = 0

for file in m_files:
    f = pd.read_csv(file)
    print(f.shape)
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round()
    f['long_km'] = (merc_x(f['longitude'])/1E3).round()
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
    
    print(f.shape)
    
    shape = shape + f.shape[0]
    print('column length', len(f.columns))
    if os.path.exists(filename):
        f.to_csv(filename, header=None, mode='a', index=False)
    else:
        f.to_csv(filename, index=False)

(4772188, 15)
(214976, 17)
column length 17
(4461419, 15)
(235249, 17)
column length 17
(4248841, 15)
(229176, 17)
column length 17
(4983109, 15)
(234913, 17)
column length 17
(4723105, 15)
(165272, 17)
column length 17
(4742107, 15)
(288160, 17)
column length 17
(4488383, 15)
(230523, 17)
column length 17
(4855536, 15)
(176189, 17)
column length 17
(5172627, 15)
(276734, 17)
column length 17
(4721882, 15)
(198856, 17)
column length 17
(5158284, 15)
(225431, 17)
column length 17
(5127336, 15)
(276406, 17)
column length 17
(5184272, 15)
(195966, 17)
column length 17
(3726003, 15)
(45055, 17)
column length 17
(185960, 15)
(7802, 17)
column length 17
(264718, 15)
(2898, 17)
column length 17
(4314389, 15)
(210563, 17)
column length 17
(4443358, 15)
(198697, 17)
column length 17
(4478281, 15)
(142868, 17)
column length 17
(4210218, 15)
(149221, 17)
column length 17
(1392967, 14)
(196963, 16)
column length 16


In [149]:
print('expected shape ', shape)
df = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m.csv')
print(df.shape)

expected shape  3901918
(3901918, 17)


In [160]:
# keep the file spot with distance 1000 km from the station
distance = 1000 # km

filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv'
shape = 0

for file in v_files:
    f = pd.read_csv(file)
    print(f.shape)
    # convert lat 
    f['lat_km'] = (f['latitude'].apply(merc_y)/1E3).round()
    f['long_km'] = (merc_x(f['longitude'])/1E3).round()
    # remove by lat 
    f = f[f['lat_km'] <= (lat_km+1000)]
    f = f[f['lat_km'] >= (lat_km-1000)]
    # remove by long 

    f = f[f['long_km'] <= (long_km+1000)]
    f = f[f['long_km'] >= (long_km-1000)]
    
    print(f.shape)
    
    shape = shape + f.shape[0]
    print('column length', len(f.columns))
    
    if os.path.exists(filename):
        f.to_csv(filename, header=None, mode='a', index=False)
    else:
        f.to_csv(filename, index=False)
        
print('final shape', shape)

(21393656, 14)
(1100046, 16)
column length 16
(20097731, 14)
(1177369, 16)
column length 16
(19319890, 14)
(1142863, 16)
column length 16
(20826285, 14)
(969884, 16)
column length 16
(16633347, 14)
(1039761, 16)
column length 16
(20299549, 14)
(1099041, 16)
column length 16
(19996363, 14)
(751954, 16)
column length 16
(19131468, 14)
(751741, 16)
column length 16
(5180494, 14)
(851508, 16)
column length 16
(1350471, 14)
(208055, 16)
column length 16
final shape 9092222


In [161]:
df = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_v.csv')
df.shape

(9092222, 16)